 Gather Data Files
Open a terminal and download the dataset:
Use the wget command to download the dataset containing multiple file formats.

Import Libraries and Set Paths
You need to import necessary libraries like:
glob to handle file formats.
pandas to read CSV and JSON files.
xml.etree.ElementTree to parse XML data.
datetime to track the progress of each phase through logging.
Install the pandas library if it's not already installed.
Set up paths for:
log_file.txt to record the logs.
transformed_data.csv to save the final output.


In [1]:
pip install pandas



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
import os

log_file_path = "log_file.txt"
output_csv_path = "transformed_data.csv"

# Create an empty log file if it doesn’t exist
if not os.path.exists(log_file_path):
    with open(log_file_path, "w") as log_file:
        log_file.write("Log File Created\n")

print("Paths set up successfully!")

Paths set up successfully!


In [10]:
log_file = "log_file.txt"

def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"{timestamp} - {message}\n"
    with open(log_file, "a") as log:
        log.write(log_entry)
    print(log_entry.strip())  # Print log to console


In [17]:
output_csv = "transformed_data.csv"

Define functions for each step of  ETL as follows:
Extract Data:
Three different functions to extract data from CSV, JSON, and XML files respectively.
A master function will call the relevant function based on the file type and combine the extracted data into a single DataFrame.
Transform Data:
The transformation process involves converting:
Heights from inches to meters.
Weights from pounds to kilograms.
This step ensures the data is in the desired format for further analysis or storage.
Load Data:
The transformed data is saved to a CSV file, which can later be loaded into a relational database or used for further processing.
Logging:
Throughout the ETL process, each phase (Extraction, Transformation, Loading) is logged with a timestamp to ensure traceability and monitoring.
Logs are saved in a text file for auditing or troubleshooting purposes.


In [11]:
file_path = r"C:\Users\ganeshraj.subramani\Downloads\source\file.csv"

def extract_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        log_message(f"Successfully extracted data from {file_path}")
        return df
    except Exception as e:
        log_message(f"Error extracting CSV {file_path}: {str(e)}")
        return pd.DataFrame()

# Extract from JSON
def extract_json(file_path):
    try:
        df = pd.read_json(file_path)
        log_message(f"Successfully extracted data from {file_path}")
        return df
    except Exception as e:
        log_message(f"Error extracting JSON {file_path}: {str(e)}")
        return pd.DataFrame()

# Extract from XML
def extract_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        data = []
        for record in root.findall("record"):
            row = {child.tag: child.text for child in record}
            data.append(row)
        df = pd.DataFrame(data)
        log_message(f"Successfully extracted data from {file_path}")
        return df
    except Exception as e:
        log_message(f"Error extracting XML {file_path}: {str(e)}")
        return pd.DataFrame()

In [12]:
def extract_data(source_folder):
    all_data = pd.DataFrame()
    for file in glob.glob(f"{source_folder}/*"):
        if file.endswith(".csv"):
            df = extract_csv(file)
        elif file.endswith(".json"):
            df = extract_json(file)
        elif file.endswith(".xml"):
            df = extract_xml(file)
        else:
            log_message(f"Skipping unknown file format: {file}")
            continue

        all_data = pd.concat([all_data, df], ignore_index=True)

    log_message("Data extraction completed.")
    return all_data


In [19]:
def transform_data(df):
    try:
        df["height"] = df["height"].astype(float) * 0.0254  # Inches to meters
        df["weight"] = df["weight"].astype(float) * 0.453592  # Pounds to kilograms
        log_message("Data transformation completed.")
        return df
    except Exception as e:
        log_message(f"Error during transformation: {str(e)}")
        return df

In [14]:
def load_data(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        log_message(f"Transformed data saved to {output_path}")
    except Exception as e:
        log_message(f"Error saving transformed data: {str(e)}")

Extraction Phase:
The project extracts data from all CSV, JSON, and XML files located in the project directory.
Each file type is processed, and the results are combined into one DataFrame.
Transformation Phase:
The extracted data undergoes transformation to convert the measurements to standard units (e.g., height to meters, weight to kilograms).
Loading Phase:
The transformed data is written into a CSV file, which can be imported into a database for further use.
Logging:
The start and end of each phase (Extraction, Transformation, Loading) are logged to track progress and ensure everything runs smoothly.


In [15]:
def run_etl(source_folder):
    log_message("ETL process started.")

    # Step 1: Extract
    extracted_data = extract_data(source_folder)

    # Step 2: Transform
    transformed_data = transform_data(extracted_data)

    # Step 3: Load
    load_data(transformed_data, output_csv)

    log_message("ETL process completed.")

In [18]:
if __name__ == "__main__":
    source_folder = "unzipped_folder"  # Change this if your files are in a different directory
    run_etl(source_folder)

2025-02-08 21:00:55 - ETL process started.
2025-02-08 21:00:55 - Data extraction completed.
2025-02-08 21:00:55 - Error during transformation: 'height'
2025-02-08 21:00:55 - Transformed data saved to transformed_data.csv
2025-02-08 21:00:55 - ETL process completed.
